# Final Project: **Location of a Vegetarian Restaurant in Mexico**

#### By: **Daniel Eduardo López**
_____




### **1. Goal**
To define the appropriate location of a Vegetarian Restaurant in Mexico City through the use of Machine Learning techniques.

### **2. Instructions**
1. Using demographic variables, perform a supervised model where the target variable is the number of restaurants.
2. Create the independent variables using the State and the ZIP code.
3. Use at least 50 variables and with some of them develop an unsupervised model uwing K-Means to profile each of the ZIP Codes.
4. Train a model using a clustering technique and the rest of the variables to predict the target Variable (use Grid Search and Cross Validation).
5. Determine 10 locations where the restaurant might be placed.
6. Determine the 5 most influential variables.
7. Present your results on Google Slides.

### **3. Code**

In [4]:
# Libraries importation
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import time
%matplotlib inline

In [2]:
# Setting of the Seaborn theme to Darkgrid
sns.set_theme(context = 'notebook', style = 'darkgrid')

In [5]:
# Setting of the INEGI's API 
token = "06bf4cf2-9d80-47a5-9317-1c5d2a59b0e7"

def radio_busq_inegi(condicion="restaurantes", radio=500, loc=[19.4339339,-99.135982]):
    lat, lng = loc
    
    places_url = f"https://www.inegi.org.mx/app/api/denue/v1/consulta/buscar/{condicion}/{lat},{lng}/{radio}/{token}"
    
    time.sleep(0.48)
    r = requests.get(places_url)
    time.sleep(0.02)
    # print(places_url, r.text)
    if r.status_code not in range(200, 300):
        return {}
    return r.json()

In [8]:
resultado = radio_busq_inegi('restaurantes',loc=[19.4339339,-99.135982])
resultado = pd.DataFrame(resultado)
resultado.head()

,CLEE,Id,Nombre,Razon_social,Clase_actividad,Estrato,Tipo_vialidad,Calle,Num_Exterior,Num_Interior,...,Ubicacion,Telefono,Correo_e,Sitio_internet,Tipo,Longitud,Latitud,CentroComercial,TipoCentroComercial,NumLocal
0,09015722514013402000000000U9,9181342,TAQUERIA ARANDAS,,Restaurantes con servicio de preparación de ta...,6 a 10 personas,AVENIDA,5 DE MAYO,46,0,...,"CUAUHTÉMOC, Cuauhtémoc, CIUDAD DE MÉXICO",,,,Fijo,-99.13578987,19.43424132,,,
1,09015722511013874000000000S5,8625789,RESTAURANTE PATANEGRA HISTORICO,PATANEGRA HISTORICO SA DE CV,Restaurantes con servicio de preparación de al...,31 a 50 personas,AVENIDA,5 DE MAYO,49,,...,"CUAUHTÉMOC, Cuauhtémoc, CIUDAD DE MÉXICO",,MALMONACIO@DOE.COM.MX,,Fijo,-99.13556320,19.43405563,,,
2,09015722514012612000000000S4,9108981,TAQUERIA TLAQUEPAQUE,,Restaurantes con servicio de preparación de ta...,6 a 10 personas,AVENIDA,ISABEL LA CATOLICA,16,,...,"CUAUHTÉMOC, Cuauhtémoc, CIUDAD DE MÉXICO",,,,Fijo,-99.13604146,19.43437677,,,
3,09015722211001591001004093S6,6795242,KFC PALMA,PREMIUM RESTAURANT BRANDS S DE RL DE CV,Restaurantes de autoservicio,11 a 30 personas,CALLE,Segunda Cerrada 5 de Mayo,19,0,...,"CUAUHTÉMOC, Cuauhtémoc, CIUDAD DE MÉXICO",,,WWW.KFC.COM.MX,Fijo,-99.13552121,19.43378746,,,
4,09015722514002891001000000U6,887890,TACOS DE GUISADO ISABELA CATOLICA,,Restaurantes con servicio de preparación de ta...,6 a 10 personas,AVENIDA,ISABEL LA CATOLICA,15,,...,"CUAUHTÉMOC, Cuauhtémoc, CIUDAD DE MÉXICO",,,,Fijo,-99.13614008,19.43442364,,,


In [ ]:


x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = 0.20)

In [ ]:

scaler = MinMaxScaler()
scaler.fit(x_train)

MinMaxScaler()

In [ ]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)